In [ ]:
# Data Analysis libraries
import pandas as pd 
import numpy as np 

#Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator

#Linear Regression libraries
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import os
import shutil  
from os import path

In [ ]:
sns.set_style("whitegrid")

In [ ]:
base = pd.read_csv("Movies_new.csv")

Describe function show that the Budget and Revenue columns has 0 as a min values, however, the info() function showed that there isnt any NaN value

In [ ]:
base.describe()
base.info()
base.count()
base.isnull().sum()

This function replace all 0 value to NaN and then it is possible to fill this values with the median of the specific column

In [ ]:
replace = base.replace(0, np.nan, inplace=True)

In [ ]:
budget_median = base.Budget.median()
revenue_median = base.Revenue.median()

In [ ]:
base["Budget"].fillna(budget_median, inplace = True)
base["Revenue"].fillna(revenue_median, inplace = True)

It's possible to save another csv with all columns complete

In [ ]:
base.to_csv("Movies_median.csv", index=False)

# Graph Analysis

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize = (12,6), sharey=True)
fig.suptitle("Comparison between User Rating and Movie Database Average Ratings", fontsize=16)
plt.subplots_adjust(wspace=0, hspace=0)
plt.ylim(1,5)
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2})
sns.despine(left= True)



user_boxplot = sns.boxplot(y = base["RATING"], 
                            color="lightskyblue", 
                            showmeans=True, 
                            meanprops= {"marker":"o",
                                        "markerfacecolor":"white", 
                                        "markeredgecolor":"black", 
                                        "markersize":"7"},
                            medianprops = {"color":"orange", 
                                           "linestyle":"--"}, 
                            ax=axs[0])


database_boxplot = sns.boxplot(y = base["Vote Average"], 
                           color="violet", 
                           showmeans=True,
                           meanprops= {"marker":"o",
                                       "markerfacecolor":"white", 
                                       "markeredgecolor":"black", 
                                       "markersize":"7"},
                           medianprops = {"color":"springgreen", 
                                          "linestyle":"--"},
                           ax=axs[1]).set(ylabel=None)

#plt.savefig("Boxplot_ratings")

# Transform the genres in a Series of genres

In [ ]:
df_genres_id= pd.Series(base["Genres"].values, index = base["ID"], name = "Genres")
split_genres = df_genres_id.str.split("|", expand = False)
genres_explode = split_genres.explode()


In [ ]:
genres_explode.unique()

In [ ]:
genres_count = genres_explode.value_counts()

# WordCloud dos generos

In [ ]:
all_genres = " ".join(s for s in genres_explode).strip()

all_genres

In [ ]:
all_genres = " ".join(s for s in genres_explode)


wordcloud = WordCloud(background_color="black",
                      font_path='Andale Mono',
                      width=1600, height=800,
                      collocations=False,
                      colormap = "viridis").generate(all_genres)

plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.tight_layout(pad=0)
plt.axis("off")

In [ ]:
wordcloud.words_

In [ ]:
palette = sns.color_palette("PiYG_r", 18)
sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 2})
sns.set(rc={'figure.figsize':(10,6)})


barplot = sns.barplot(genres_count.index,
                      genres_count.values,
                      palette = palette
                      )

plt.xticks(rotation=45, fontsize=12, ha = "right")
plt.yticks(fontsize=12)

# Top 10 voted movies

In [ ]:
vote_count = base.sort_values("Vote Count", ascending= False)
top_voted = vote_count.head(n =10)

top_voted

In [ ]:
palette = sns.color_palette("Purples_r", 10)
sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 2})
sns.set(rc={'figure.figsize':(10,6)})


barplot = sns.barplot(x= top_voted["Vote Count"],
                      y = top_voted["MOVIE"], 
                      palette = palette,
                      orient = "h").set(ylabel = None)

plt.xticks(rotation=45, fontsize=12, ha = "center")
plt.yticks(fontsize=12)


#img = plt.savefig("Barplot_votecount", bbox_inches='tight')

# Finding the linear correlation between Budget and Revenue

* Trying to discover if it is possible to predict the Revenue by the Budget

In [ ]:
ax = sns.scatterplot(x=base['Budget'], y=base['Revenue'])  
#ax.set_yscale('log')
#ax.set_xscale('log')

X_train, X_test, y_train, y_test = train_test_split(base.Budget, base.Revenue)

plt.scatter(X_train,y_train, label = "Training Data", color = "r", alpha= .7)
plt.scatter(X_test,y_test, label = "Testing Data", color = "g", alpha= .7)
plt.legend()
#plt.yscale('log')
#plt.xscale('log')


lr = LinearRegression()
lr.fit(X_train.values.reshape(-1,1), y_train.values)

prediction = lr.predict(X_test.values.reshape(-1,1))
plt.plot(X_test,prediction,label = "Linear Regression", color = "b")
plt.scatter(X_test,y_test, label = "Actual Test Data", color = "g", alpha=.7)
plt.legend()
#plt.yscale('log')
#plt.xscale('log')

sns.distplot(base['Budget'])

corre = np.corrcoef(base["Budget"],base["Revenue"])

In [ ]:
ax = sns.scatterplot(x=base['Budget'], y=base['Revenue'])  
#ax.ticklabel_format(style='sci',scilimits=(4,9),axis='both')

#plt.rcParams['axes.formatter.limits'] = (4, 9)
ax.axes.set_ylim(min(base.Budget), max(base.Revenue))


In [ ]:
base.describe()